In [ ]:
import os
import cv2
import face_recognition
import numpy as np
import datetime

# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Provide the full path to your main images folder
main_image_folder = r'C:\Users\USER\Documents\images' 

# Check if the folder exists
if os.path.exists(main_image_folder):
    print("Main image folder found!")
else:
    print("Main image folder not found. Please check the path.")

known_face_encodings = []
known_face_names = []

# Iterate over subfolders in the main image folder
for person_name in os.listdir(main_image_folder):
    person_folder = os.path.join(main_image_folder, person_name)

    # Check if it is a directory (subfolder for each person)
    if os.path.isdir(person_folder):
        # Iterate over each image in the person's subfolder
        for image_file in os.listdir(person_folder):
            if image_file.endswith(".jpg") or image_file.endswith(".png"):
                # Load the image
                image_path = os.path.join(person_folder, image_file)
                image = face_recognition.load_image_file(image_path)

                # Attempt to get the face encoding
                encodings = face_recognition.face_encodings(image)

                # Check if any faces were found
                if len(encodings) > 0:
                    image_encoding = encodings[0]  # Take the first encoding if available
                    known_face_encodings.append(image_encoding)
                    known_face_names.append(person_name)  # Use the subfolder name as the person's name
                else:
                    print(f"No face found in image {image_file}. Skipping this image.")

print("Images loaded and encoded successfully.")

# Save encodings to file
np.save('encodings.npy', known_face_encodings)
np.save('names.npy', known_face_names)

# Set to track already marked attendance
attendance_marked = set()

# Function to mark attendance
def mark_attendance(name):
    # Mark attendance only if it's not already marked for the person
    if name not in attendance_marked:
        if not os.path.exists("Attend.csv"):
            with open("Attend.csv", 'w') as f:
                f.writelines("Name,Timestamp\n")  # Write CSV headers

        with open("Attend.csv", 'r+') as f:
            mydatalist = f.readlines()
            name_list = []
            for line in mydatalist:
                entry = line.split(',')
                name_list.append(entry[0])
            if name not in name_list:
                now = datetime.datetime.now()
                f.writelines(f'{name},{str(now)}\n')  # Mark attendance
                print(f'Attendance marked for {name}')
                # Add to the set to prevent duplicate marking
                attendance_marked.add(name)

# Initialize webcam for face recognition
cap = cv2.VideoCapture(0)

# Add a frame skip counter to reduce the processing load
frame_skip = 3
frame_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to capture video.")
            break

        frame_count += 1
        if frame_count % frame_skip != 0:  # Skip frames to reduce load
            continue

        # Resize frame for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Find all face locations and face encodings in the current frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        # Loop through each face found in the frame
        for face_encoding, face_location in zip(face_encodings, face_locations):
            if len(known_face_encodings) > 0:
                # Check if the face matches any known faces
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

                if len(face_distances) > 0:
                    best_match_index = np.argmin(face_distances)

                    if matches[best_match_index] and face_distances[best_match_index] < 0.4:  # Threshold for recognition
                        name = known_face_names[best_match_index].upper()

                        # Mark attendance only once for the person
                        mark_attendance(name)

                        # Draw a box around the face
                        top, right, bottom, left = face_location
                        top, right, bottom, left = top * 4, right * 4, bottom * 4, left * 4
                        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

                        # Label the face with a name
                        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
                    else:
                        # Draw a box around the face for unknown faces
                        top, right, bottom, left = face_location
                        top, right, bottom, left = top * 4, right * 4, bottom * 4, left * 4
                        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                        # Label face as 'Not Matched'
                        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                        cv2.putText(frame, 'Not Matched', (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)

        # Display the resulting frame
        cv2.imshow('Webcam Face Recognition', frame)

        # Exit on pressing 'x'
        if cv2.waitKey(1) & 0xFF == ord('x'):
            print("Exiting...")
            break

except KeyboardInterrupt:
    print("\nProcess interrupted by user. Exiting...")

finally:
    # Release the webcam and close windows
    cap.release()
    cv2.destroyAllWindows()


Main image folder found!
No face found in image cd3.jpg. Skipping this image.
No face found in image dinesh3.jpg. Skipping this image.
No face found in image divi2.jpg. Skipping this image.
No face found in image divi3.jpg. Skipping this image.
No face found in image m1.jpg. Skipping this image.
No face found in image m2.jpg. Skipping this image.
No face found in image m4.jpg. Skipping this image.
No face found in image nm2.jpg. Skipping this image.
No face found in image nm3.jpg. Skipping this image.
No face found in image prem.jpg. Skipping this image.
No face found in image dv.jpg. Skipping this image.
Images loaded and encoded successfully.
Attendance marked for NANI
Attendance marked for SURYA
Attendance marked for DINESH KUMAR
Attendance marked for VIJAY
